In [64]:
import pandas as pd

In [65]:
df = pd.read_csv('./cleaned_airbnb_data.csv')

In [66]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,price_category,length_of_stay_category
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365,Medium,Low
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355,Medium,Low
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaT,NaN,1,365,Medium,Low
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194,Low,Low
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0,Low,Low


In [67]:
df.shape

(48884, 18)

1

Use the pivot_table function to create a detailed summary that reveals the average price for different combinations of neighbourhood_group and room_type. This analysis will help identify high-demand areas and optimize pricing strategies across various types of accommodations (e.g., Entire home/apt vs. Private room).

In [68]:
pd.pivot_table(data=df, index='neighbourhood_group', columns='room_type', values='price', aggfunc='mean')

room_type,Entire home/apt,Private room,Shared room
neighbourhood_group,,,
Bronx,127.506596,66.890937,59.800000
Brooklyn,178.346202,76.545428,50.773723
Manhattan,249.257994,116.776622,88.977083
Queens,147.050573,71.762456,69.020202
Staten Island,173.846591,62.292553,57.444444


Transform the dataset from a wide format to a long format using the melt function. This restructuring facilitates more flexible and detailed analysis of key metrics like price and minimum_nights, enabling the identification of trends, outliers, and correlations.

In [69]:
pd.melt(df, id_vars=['name'], value_vars=['price', 'minimum_nights'], 
                  var_name='metric', value_name='value')

,name,metric,value
0,Clean & quiet apt home by the park,price,149
1,Skylit Midtown Castle,price,225
2,THE VILLAGE OF HARLEM....NEW YORK !,price,150
3,Cozy Entire Floor of Brownstone,price,89
4,Entire Apt: Spacious Studio/Loft by central park,price,80
...,...,...,...
97763,Charming one bedroom - newly renovated rowhouse,minimum_nights,2
97764,Affordable room in Bushwick/East Williamsburg,minimum_nights,4
97765,Sunny Studio at Historical Neighborhood,minimum_nights,10
97766,43rd St. Time Square-cozy single bed,minimum_nights,1


Create a new column availability_status using the apply function, classifying each listing into one of three categories based on the availability_365 column:

- "Rarely Available": Listings with fewer than 50 days of availability in a year.
- "Occasionally Available": Listings with availability between 50 and 200 days.
- "Highly Available": Listings with more than 200 days of availability.

In [70]:
def set_availiability_status(n_days):
    if n_days < 50:
        return 'Rarely Available'
    elif 50 <= n_days < 200:
        return 'Occasionally Available'
    else:
        return 'Highly Available'

In [71]:
df['availability_status'] = df['availability_365'].apply(set_availiability_status)
df.tail()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,price_category,length_of_stay_category,availability_status
48879,36484665,Charming one bedroom - newly renovated rowhouse,8232441,Sabrina,Brooklyn,Bedford-Stuyvesant,40.67853,-73.94995,Private room,70,2,0,NaT,NaN,2,9,Low,Low,Rarely Available
48880,36485057,Affordable room in Bushwick/East Williamsburg,6570630,Marisol,Brooklyn,Bushwick,40.70184,-73.93317,Private room,40,4,0,NaT,NaN,2,36,Low,Low,Rarely Available
48881,36485431,Sunny Studio at Historical Neighborhood,23492952,Ilgar & Aysel,Manhattan,Harlem,40.81475,-73.94867,Entire home/apt,115,10,0,NaT,NaN,1,27,Medium,Low,Rarely Available
48882,36485609,43rd St. Time Square-cozy single bed,30985759,Taz,Manhattan,Hell's Kitchen,40.75751,-73.99112,Shared room,55,1,0,NaT,NaN,6,2,Low,Low,Rarely Available
48883,36487245,Trendy duplex in the very heart of Hell's Kitchen,68119814,Christophe,Manhattan,Hell's Kitchen,40.76404,-73.98933,Private room,90,7,0,NaT,NaN,1,23,Low,Low,Rarely Available


Analyze trends and patterns using the new availability_status column, and investigate potential correlations between availability and other key variables like price, number_of_reviews, and neighbourhood_group to uncover insights that could inform marketing and operational strategies.

In [72]:
df.groupby(['availability_status', 'neighbourhood_group'])[['price', 'number_of_reviews']].aggregate(['min', 'mean', 'median', 'max'])

price                            \
                                             min        mean median    max   
availability_status    neighbourhood_group                                   
Highly Available       Bronx                  25   93.367788   69.0    800   
                       Brooklyn               11  139.993303   99.0   8000   
                       Manhattan              11  242.928159  179.0   9999   
                       Queens                 10  107.966199   75.0   2600   
                       Staten Island          20  124.772021   75.0   5000   
Occasionally Available Bronx                  10   86.959239   65.0   2500   
                       Brooklyn               10  131.114299   98.0   7500   
                       Manhattan              10  209.732504  150.0  10000   
                       Queens                 10   96.518318   75.0   1800   
                       Staten Island          29   93.102804   75.0    450   
Rarely Available       Bronx                  20   80.447712   65.0    680   
                       Brooklyn               10  115.106528   89.0  10000   
                       Manhattan              10  167.494139  130.0   9999   
                       Queens                 16   94.102502   74.5  10000   
                       Staten Island          13  120.301370   75.0   1250   

                                           number_of_reviews             \
                                                         min       mean   
availability_status    neighbourhood_group                                
Highly Available       Bronx                               0  32.490385   
                       Brooklyn                            0  41.904772   
                       Manhattan                           0  27.998514   
                       Queens                              0  37.093084   
                       Staten Island                       0  39.906736   
Occasionally Available Bronx                               0  24.163043   
                       Brooklyn                            0  32.735013   
                       Manhattan                           0  31.821982   
                       Queens                              0  32.021021   
                       Staten Island                       0  24.130841   
Rarely Available       Bronx                               0  19.307190   
                       Brooklyn                            0  13.209066   
                       Manhattan                           0  13.178814   
                       Queens                              0  15.546198   
                       Staten Island                       0  17.219178   

                                                        
                                           median  max  
availability_status    neighbourhood_group              
Highly Available       Bronx                 12.0  291  
                       Brooklyn              16.0  488  
                       Manhattan              5.0  607  
                       Queens                13.0  629  
                       Staten Island         20.0  333  
Occasionally Available Bronx                  8.0  206  
                       Brooklyn              13.0  467  
                       Manhattan             10.0  540  
                       Queens                12.0  576  
                       Staten Island         10.0  136  
Rarely Available       Bronx                  5.5  321  
                       Brooklyn               3.0  480  
                       Manhattan              3.0  451  
                       Queens                 3.0  424  
                       Staten Island          2.0  147

2

Perform basic descriptive statistics (e.g., mean, median, standard deviation) on numeric columns such as price, minimum_nights, and number_of_reviews to summarize the dataset's central tendencies and variability, which is crucial for understanding overall market dynamics.

In [73]:
df[['price', 'minimum_nights', 'number_of_reviews']].describe()

,price,minimum_nights,number_of_reviews
count,48884.000000,48884.000000,48884.000000
mean,152.755053,7.029887,23.271991
std,240.170260,20.512224,44.551331
min,10.000000,1.000000,0.000000
25%,69.000000,1.000000,1.000000
50%,106.000000,3.000000,5.000000
75%,175.000000,5.000000,24.000000
max,10000.000000,1250.000000,629.000000


3

Convert the last_review column to a datetime object and set it as the index of the DataFrame to facilitate time-based analyses.

In [74]:
df['last_review'] = pd.to_datetime(df['last_review'])
df['month'] =  df['last_review'].dt.month

df.set_index('last_review', inplace=True)

Resample the data to observe monthly trends in the number of reviews and average prices, providing insights into how demand and pricing fluctuate over time.

In [75]:
monthly_trends = df.resample('M').aggregate({
    'price': 'mean',
    'number_of_reviews': 'sum'
}).rename(columns={'price': 'avg_price', 'reviews': 'total_reviews'})

monthly_trends.tail(50)

C:\Users\t-bva\AppData\Local\Temp\ipykernel_12552\780865841.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_trends = df.resample('M').aggregate({


,avg_price,number_of_reviews
last_review,,
2015-06-30,143.911111,307
2015-07-31,121.794393,259
2015-08-31,136.900498,707
2015-09-30,136.902326,830
2015-10-31,171.674107,1228
2015-11-30,134.445205,728
2015-12-31,158.906103,896
2016-01-31,191.074713,1518
2016-02-29,246.540541,369


Group the data by month to calculate monthly averages and analyze seasonal patterns, enabling better forecasting and strategic planning around peak periods.

In [77]:
df.groupby('month').aggregate({
    'price': 'mean',
    'number_of_reviews': 'sum'
}).rename(columns={'price': 'avg_price', 'reviews': 'total_reviews'})


,avg_price,number_of_reviews
month,,
1.0,150.587514,31041
2.0,155.716883,9472
3.0,136.836957,15032
4.0,139.504979,30588
5.0,137.968291,94850
6.0,140.821481,628230
7.0,142.290382,253512
8.0,132.133173,12832
9.0,149.494463,14243
